# 🍏 Health & Fitness Agent with Bing Grounding 🍎

Welcome to our **Health & Fitness Agent with Bing Grounding** tutorial! In this notebook, we'll demonstrate how to:

1. **Initialize** a project using Azure AI Foundry.
2. **Create an Agent** with the **BingGroundingTool** for web search.
3. **Ask real-world questions** about health and fitness.
4. **Retrieve and display** answers, including Bing query URLs and disclaimers.

### ⚠️ Important Model Support Note ⚠️
> Bing grounding is currently only supported in certain Azure OpenAI models (e.g. `gpt-4o-0513`).
> 
> Make sure you specify a supported model and set the `"x-ms-enable-preview": "true"` header.

## Prerequisites
- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- Grounding with Bing connection in Azure AI Foundry, which has to be provisioned from Azure portal.
See ["Setup Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) in the documentation for full details.
<img src="./seq-diagrams/bing-api-key.png" width="30%"/>
- A `.env` file in the parent directory containing:
  ```bash
  PROJECT_CONNECTION_STRING=<your-connection-string>
  MODEL_DEPLOYMENT_NAME=<supported-model>
  BING_CONNECTION_NAME=<the-name-of-your-bing-connection>
  ```

## Let's Explore Bing Grounding!
We'll integrate **Bing** search results into our agent so it can gather extra context from the web. We'll store and display the Bing search query link for transparency. 🎉

<img src="./seq-diagrams/4-bing-grounding.png" width="30%"/>


## 1. Initial Setup
We'll load environment variables from `.env` and initialize our **AIProjectClient** to manage agents.

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool, MessageTextContent

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING")
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

## 2. Create Bing-Grounded Agent 🌐
We'll fetch our Bing connection from AI Foundry and use `BingGroundingTool` to let our agent search the web. Then we'll create a new agent with disclaimers about not being a doctor, etc.

Make sure your `MODEL_DEPLOYMENT_NAME` is set to a Bing-supported model (for example, `gpt-4o-0513`) and that you add the header `{"x-ms-enable-preview": "true"}`.

In [ ]:
def create_bing_grounded_agent():
    """Create an agent that can use Bing to ground queries with up-to-date info."""
    try:
        # 1. Retrieve Bing connection from your AI Foundry project
        bing_conn_name = os.environ.get("BING_CONNECTION_NAME")
        if not bing_conn_name:
            raise ValueError("BING_CONNECTION_NAME not set in .env")

        bing_connection = project_client.connections.get(connection_name=bing_conn_name)
        conn_id = bing_connection.id
        print(f"Bing Connection ID: {conn_id}")

        # 2. Initialize Bing grounding tool
        bing_tool = BingGroundingTool(connection_id=conn_id)

        # 3. Create an agent that can search with Bing
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-0513"),
            name="health-bing-agent",
            instructions="""
                You are a health and fitness assistant with Bing search capabilities.
                Always:
                1. Provide disclaimers that you are not a medical professional.
                2. Encourage professional consultation.
                3. Use Bing for real-time references.
                4. Provide brief, helpful answers.
            """,
            tools=bing_tool.definitions,
            # Must pass special preview header to use Bing grounding (subject to change)
            headers={"x-ms-enable-preview": "true"},
        )

        print(f"🎉 Created Bing-grounded agent, ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating Bing-grounded agent: {e}")
        return None

# Create our Bing-based agent
bing_agent = create_bing_grounded_agent()

## 3. Starting Threads & Asking Questions 💬
We'll create conversation threads for each user query, letting the agent search with Bing to find relevant info. We will store all `(thread, run)` pairs in a list so we can review them in the next step.

In [ ]:
bing_threads = []

def ask_bing_question(agent, user_query):
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created a conversation thread, ID: {thread.id}")

        # Post user query as a message
        user_message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_query
        )
        print(f"📨 Created user message with query: '{user_query}'")

        # Process the query with the agent
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 Run finished with status: {run.status}\n")
        if run.last_error:
            print(f"Error detail: {run.last_error}\n")

        return thread, run
    except Exception as e:
        print(f"❌ Error asking Bing question: {e}")
        return None, None

if bing_agent:
    # We'll ask a few fun questions!
    questions = [
        "What are some new HIIT workout trends I should know about?",
        "What's the current WHO recommendation for sugar intake?",
        "Any news on intermittent fasting for weight management?"
    ]

    for q in questions:
        thr, rn = ask_bing_question(bing_agent, q)
        if thr and rn:
            bing_threads.append((thr, rn))

## 4. Viewing Bing-Grounded Answers & Query URLs
We’ll retrieve each thread's messages, printing both the user queries and the agent's responses. We'll also fetch the run steps to display the **Bing Search Query URL** (so you can comply with the requirement to show where the data came from). You can replace `api.bing.microsoft.com` with `www.bing.com` to form a user-friendly link.

Because `RunStep` objects do **not** have `.details`, we look instead for `'request_url'` in `step["parameters"]`. If found, it's presumably the Bing step.

In [ ]:
def view_bing_conversation(thread_id, run_id):
    try:
        # Print conversation
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation for thread:", thread_id)
        reversed_msgs = list(reversed(messages.data))  # oldest first
        for msg in reversed_msgs:
            role = msg.role.upper()
            if msg.content:
                for c in msg.content:
                    if hasattr(c, 'text') and c.text:
                        print(f"{role}: {c.text.value}\n")

        # Retrieve run steps to get Bing search query link
        run_steps = project_client.agents.list_run_steps(thread_id=thread_id, run_id=run_id)
        steps_data = run_steps.get('data', [])
        if steps_data:
            print("\n🔎 Bing run steps:")
            for step in steps_data:
                # 'parameters' is typically where BingGroundingTool stores request_url
                params = step.get('parameters', {})
                if 'request_url' in params:
                    original_url = params['request_url']
                    friendly_url = original_url.replace("api.bing.microsoft.com", "www.bing.com")
                    print(f"    Bing search URL: {friendly_url}")
        else:
            print("No run step data found for this conversation.")
    except Exception as e:
        print(f"❌ Error viewing Bing conversation: {e}")

# Display all queries and agent responses
if bing_threads:
    for (thr, rn) in bing_threads:
        view_bing_conversation(thr.id, rn.id)

## 5. Cleanup & Best Practices
You can optionally delete the agent once you're done. In production, you might keep it around for repeated usage.

### Best Practices
1. **Accuracy** – Bing search results may include disclaimers or partial info. Encourage verification with credible sources.
2. **Bing Query Display** – For compliance with Bing's use and display requirements, show both **website URLs** (in the agent's response) and **Bing search query URLs** (shown above). If the model includes citations, display them as well.
3. **Limits** – Keep an eye on usage, rate limits, or policy constraints for Bing.
4. **Privacy** – Filter search queries to avoid sending sensitive data.
5. **Evaluations** – Use `azure-ai-evaluation` for iterative improvement.


In [ ]:
def cleanup_bing_agent(agent):
    if agent:
        try:
            project_client.agents.delete_agent(agent.id)
            print(f"🗑️ Deleted Bing-grounded agent: {agent.name}")
        except Exception as e:
            print(f"❌ Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")

# Uncomment if you want to remove the agent now
cleanup_bing_agent(bing_agent)

# Congratulations! 🎉
You've built a **Bing-Grounded Health & Fitness Agent** that can:
1. **Search** the web with Bing.
2. **Answer** health/fitness questions with disclaimers.
3. **Include** references and Bing search query links.

Feel free to expand this approach by combining the BingGroundingTool with other tools (e.g., **FileSearchTool**, **CodeInterpreterTool**) to build a robust advisor.

#### Let's proceed to [5-agents-aisearch.ipynb](./5-agents-aisearch.ipynb)